# Import packages and start docker

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from collections import Counter
import docker
import re
import pandas as pd
import random

c:\Users\janva\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


to do
-add link to consumer (maybe for experience measures)
-add link to video instead of yes/no
-add a list of links of pictures/videos from the product description


london -006
london -002

In [2]:
client = docker.from_env()
container = client.containers.get('splash')
#start container
container.start()

In [3]:
#read json file C:\Users\janva\OneDrive\Documenten\Tilburg MSc jaar 3\Block 2\Machine Learning\useragent-data.json
import json
with open('C:\\Users\\janva\\OneDrive\\Documenten\\Tilburg MSc jaar 3\\Block 2\\Machine Learning\\useragent-data.json') as f:
    user_agents = json.load(f)
#get the user_agents column
user_agents = [u['userAgent'] for u in user_agents]
#get random user agent randomly
def get_random_user_agent():
    return user_agents[random.randint(0, len(user_agents) - 1)]

In [4]:
def get_soup(url):
    t = 0
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 0.2
    else:
        time_wait = 0.2
    while True:
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'viewport': '320x480', 'images': 0, 'resource_timeout': 10}, headers={'User-Agent': get_random_user_agent()})
        #r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'resource_timeout': 10}, headers={'User-Agent': get_random_user_agent()})
        #request without docker
        #r = requests.get(url, headers={'User-Agent': get_random_user_agent()})
        soup = BeautifulSoup(r.text, "html.parser")

        #check if h4 is nonetype
        if soup.find('h4') == None:
            pass
        elif soup.find('h4').text == 'Enter the characters you see below':
            raise Exception('Captcha')
        #print(soup.title.text)
        if soup.text.find('GlobalTimeoutError') != -1:
            container.restart()
            time.sleep(31)
            print(f"time sleep try {t}")
            time_wait += 4
            if t == 2:
                print(f"too many timeouts (2) for url: {url}")
                break
            t += 1
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")
            break       
    return soup

# Product url scraping

In [21]:
def get_link_asin(link):
    link_asin = {}
    splitted_link = link['href'].split('/')
    name = splitted_link[1]
    link_asin['link'] = name
    asin = splitted_link[3]
    link_asin['asin'] = asin
    #You can create the link to the review page with the name and asin
    link_asin['full_link'] = "https://www.amazon.com/"+str(name)+"/product-reviews/"+str(asin)+"/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="
    return link_asin

In [9]:
#with open(f"../../gen/input/amazon_links_per_category.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["name", "asin", "product_link", "page_link", "category"])


In [22]:
all_product_links = []
category = "computer adapter"
i = 12
with open(f"../../gen/input/amazon_links_per_category.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    while i < 24:
        page_link = f"https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page={i}&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_{i}"
        soup = get_soup(page_link)
        print(f"currently looking at page {i}")
        #Block usually after about 100 pages
        if soup.title.text=="Sorry! Something went wrong!":
            print("Blocked")
            break
        #If not block, gather asin and link
        else:
            links = soup.find_all(class_="a-link-normal s-no-outline")
        for link in links:
            if link['href'].find('/dp/') != -1 and not link['href'].startswith('/dp/'):
                link_asin = get_link_asin(link)
                link_asin['page_link'] = page_link
                link_asin['category'] = category
                all_product_links.append(link_asin)
                writer.writerow(list(link_asin.values()))
        #Try to find next page button. If absent, break
        try:
            soup.find('span', class_ = "s-pagination-item s-pagination-next s-pagination-disabled").text
            print('last page')
            break
        except:
            pass
        print(len(all_product_links))
        i += 1
    #stop docker
    container.stop()
    container.start()

page successfully loaded in time 0.2: https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page=12&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_12
currently looking at page 12
31
page successfully loaded in time 0.2: https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page=13&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_13
currently looking at page 13
62
page successfully loaded in time 0.2: https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page=14&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_14
currently looking at page 14
95
page successfully loaded in time 0.2: https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2C

# Review scraping

In [51]:
#with open(f"../../data/amazon_search_hedonic.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified", "profile", "video", "scrape_date", "profile_url"])

In [5]:
#This function retrieves all the review info from a page
def get_reviews_from_page(product_info, start=1, end = 500):
    print(f"succesfully loaded {product_info['title']}")
    with open(f"../../data/amazon_search_hedonic.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        for i in range(start,end):
            missing_page = []
            soup_reviews = get_soup((product_info['link']+str(i)).replace("ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=", f"ref=cm_cr_getr_d_paging_btm_next_{i}?pageNumber="))
            
            #check if chaptcha
            if soup_reviews.find('h4') == None:
                pass
            elif soup_reviews.find('h4').text == 'Enter the characters you see below':
                print('chaptcha check')
                #container.stop()
                raise Exception('chaptcha')
            reviews = soup_reviews.find_all('div', {"data-hook":"review"})
            if len(reviews) == 0:
                print(f'Could not find any reviews at page {i}')
                break
            else:
                pass



            for review in reviews:
                review_info = {}
                review_info = {
                'product_url' : product_info['link']+str(i),
                'brand' : product_info['brand'],
                'model' : product_info['model'],
                'link' : product_info['link']+str(i),
                'product_title' : product_info['title']
                }
                try:
                    review_info['name'] = review.find('span', class_="a-profile-name").text
                except:
                    review_info['name'] = None
                try:
                    review_info['star_rating'] = float(review.find('i', {"data-hook":"review-star-rating"}).text.replace(' out of 5 stars',''))
                except:
                    #foreign countries
                    try:
                        review_info['star_rating'] = float(review.find('i', {"data-hook":"cmps-review-star-rating"}).text.replace(' out of 5 stars',''))
                    except:
                        review_info['star_rating'] = None
                try:
                    review_info['title'] = review.find('a', {"data-hook":"review-title"}).text.strip()
                except:
                    try:
                        #foreign countries
                        review_info['title'] = review.find('span',{"data-hook":"review-title"}).text.strip()
                    except:
                        review_info['title'] = None
                try:
                    review_info['country'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[0]
                except:
                    review_info['country'] = None
                try:
                    review_info['date'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[2].strip()
                except:
                    review_info['date'] = None
                attr = []
                try:
                    attributes = str(reviews[0].find('a', {"data-hook":"format-strip"}))
                    for item in re.findall(r'[>](.+?)[<]', attributes):
                        attr.append(item.replace('</i>',''))
                    review_info['info'] = attr
                except:
                    review_info['info'] = None
                try:
                    for at in attr:
                        if at.find("Color:") != -1:
                            color = at.replace("Color: ", "")
                    review_info['color'] = color
                except:
                    review_info['color'] = None
                try:
                    span_elements = review.find('span', {"data-hook":"review-body"}).find_all('span')
                    filtered_span_elements = [span for span in span_elements if span.get('class') is None]
                    concatenated_text = ''.join([span.text for span in filtered_span_elements])
                    review_info['review'] = concatenated_text
                except:
                    review_info['review'] = None
                try:
                    helpful = review.find('span', {"data-hook" : "helpful-vote-statement"}).text.strip()
                    if helpful == "One person found this helpful":
                        review_info['helpful'] = 1
                    elif "found this helpful" in helpful:
                        review_info['helpful'] = helpful.split()[0]
                    else:
                        review_info['helpful'] = helpful
                except:
                    review_info['helpful'] = None
                try:
                    #get links for consumer pictures
                    if review.find('div', class_="review-image-tile-section") is None:
                        review_info['picture'] = 'no'
                    else:
                        pics = review.find('div', class_ = "a-section a-spacing-top-mini cr-lightbox-image-thumbnails").find_all('img')
                        pics = [img['src'] for img in pics]
                        review_info['picture'] = pics
                        #saving pics
                        try:
                            for pic in pics:
                                hq_pic = pic.replace("_SY88.jpg", "_SL1600_.jpg")
                                r = requests.get(hq_pic)
                                with open(f"../../gen/output/consumer_photos_category/"+hq_pic.split('/')[-1], "wb") as f:
                                    f.write(r.content)
                                time.sleep(0.1)
                        except:
                            print("Error while downloading pictures")
                except:
                    review_info['picture'] = None

                try:
                    if review.find('span', {"data-hook":"avp-badge"}) is None:
                        review_info['verified'] = 'no'
                    else:
                        review_info['verified'] = 'yes'
                except:
                    review_info['verified'] = None
                    
                try:
                    pf_link = review.find('div', class_="a-profile-avatar").find('img')['data-src']
                    #check if it is a default image
                    if pf_link.find('default') != -1:
                        pf_link = None
                    else:
                        #saving picture
                        try:
                            pf_link = pf_link.replace("_SX48_.jpg", "_UX460_.jpg")
                            r = requests.get(pf_link)
                            with open(f"../../gen/output/profile_photos_category/"+pf_link.split('/')[-1], "wb") as f:
                                f.write(r.content)
                        except:
                            print('error while downloading profile picture')            
                    review_info['profile_picture'] = pf_link

                except:
                    review_info['profile_picture'] = None
                    
                try:
                    review_info['video'] = review.find('input', class_="video-url").get('value')
                except:
                    review_info['video'] = None
                review_info['scrape_date'] = time.strftime("%Y-%m-%d %H:%M:%S")
                try:
                    review_info['profile_url'] = review.find('a', class_="a-profile").get('href')
                except:
                    review_info['profile_url'] = None
                all_reviews.append(review_info)
                missing_values = [key for key,value in review_info.items() if value == None]
                missing_page += missing_values
                writer.writerow(list(review_info.values()))
            print(len(all_reviews))
            if len(all_reviews) % 500 == 0:
                print('restart container')
                container.restart()
                time.sleep(15)
            #missing values checker
            if len(missing_page) > 0:
                print(f"the following items were missing: {dict(Counter(missing_page))}")

            if not soup_reviews.find('li', class_ = "a-disabled a-last"):
                pass
            else:
                print(f"page {i} was the last page")
                break

In [30]:
#load csv using pandas
import pandas as pd
import re
all_reviews = []
#no headers
links_data = pd.read_csv(f"../../gen/input/amazon_links_per_category.csv", delimiter = ";")
links_data

,name,asin,product_link,page_link,category
0,Sony-DVPSR510H-Player-HDMI-Upscaling,B007F9XHAY,https://www.amazon.com/Sony-DVPSR510H-Player-H...,https://www.amazon.com/s?k=dvd+player&i=electr...,dvd_player
1,Craig-Compact-Player-Remote-CVD512a,B00A71YOQI,https://www.amazon.com/Craig-Compact-Player-Re...,https://www.amazon.com/s?k=dvd+player&i=electr...,dvd_player
2,Player-Players-Included-Up-Convert-Breakpoint,B07Z7YMQYT,https://www.amazon.com/Player-Players-Included...,https://www.amazon.com/s?k=dvd+player&i=electr...,dvd_player
3,Player-Foramor-Support-Control-Players,B07V82TGYJ,https://www.amazon.com/Player-Foramor-Support-...,https://www.amazon.com/s?k=dvd+player&i=electr...,dvd_player
4,Player-Upscaling-Coaxial-Output-Included,B099DX4RY3,https://www.amazon.com/Player-Upscaling-Coaxia...,https://www.amazon.com/s?k=dvd+player&i=electr...,dvd_player
...,...,...,...,...,...
7582,Female-Adapter-Smartphones-Tablets-Laptops,B0D1BBPHJB,https://www.amazon.com/Female-Adapter-Smartpho...,https://www.amazon.com/s?k=computer+adapter&i=...,computer adapter
7583,Female-Adapter-Smartphones-Tablets-Laptops,B0D1BHRXHC,https://www.amazon.com/Female-Adapter-Smartpho...,https://www.amazon.com/s?k=computer+adapter&i=...,computer adapter
7584,MoKo-Lightning-Supports-Charging-Transfer,B0CRRQZFRL,https://www.amazon.com/MoKo-Lightning-Supports...,https://www.amazon.com/s?k=computer+adapter&i=...,computer adapter
7585,MoKo-USB-Lightning-Adapter-Headphones,B0CRRRTKZN,https://www.amazon.com/MoKo-USB-Lightning-Adap...,https://www.amazon.com/s?k=computer+adapter&i=...,computer adapter


In [19]:
links_data[links_data['category'] == 'computer adapter']['page_link'][1484]

'https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page=11&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_11'

In [31]:
def split_join_name(name):
    return '-'.join(name.split('-')[0:3])

links_data['name_part'] = links_data['name'].apply(split_join_name)
#remove duplicates in name_part
print(len(links_data))
links_data = links_data.drop_duplicates(subset='name_part', keep='first')
print(len(links_data))

7587
4234


In [82]:
data_so_far = pd.read_csv(f"../../data/amazon_search_hedonic.csv", delimiter = ";")
data_so_far['product_title_clean'] = data_so_far['product_title'].apply(split_join_name)
#only keep links_data rows which name_part is not in product_title_clean
links_data = links_data[~links_data['name_part'].isin(data_so_far['product_title_clean'])]
print(len(links_data))

1768


In [83]:
len(data_so_far)

166392

In [40]:
def remove_last_digit(string):
    return re.sub(r'\d+$', '', string)


data_so_far['asin_url_clean'] = data_so_far['asin_url'].apply(remove_last_digit)
#merge with links_data
links_data_full = pd.read_csv(f"../../gen/input/amazon_links_per_category.csv", delimiter = ";")
data_so_far = pd.merge(data_so_far, links_data_full, left_on = 'asin_url_clean', right_on = 'product_link', how = 'left')
print(data_so_far['category'].value_counts())
print("----------------------")
#remove duplicates for link, product_title, review_title, review, date, name_x
data_so_far = data_so_far.drop_duplicates(subset = ['review_title', 'review', 'date', 'name_x', 'profile_url'])

#remove obs not from the united states
data_so_far = data_so_far[data_so_far['country'] == 'the United States']
data_so_far = data_so_far[data_so_far['review'] != '']
#show number of observations per category_y
print(data_so_far['category'].value_counts())

Digital Photo Frames    57397
smartphone              38272
computer adapter        20281
vacuum cleaner          16511
mascara                 15233
microwave oven          15192
party dress             12679
printer                 12524
lipstick                12083
razor                   11878
electric toothbrush     11779
dvd_player              10590
Name: category, dtype: int64
----------------------
mascara                 12525
microwave oven          10984
vacuum cleaner          10876
razor                   10802
Digital Photo Frames    10786
lipstick                10482
computer adapter        10275
electric toothbrush      9872
party dress              9682
smartphone               9603
dvd_player               9337
printer                  9281
Name: category, dtype: int64


In [49]:
data_so_far.columns

Index(['asin_url', 'brand', 'model', 'link', 'product_title', 'name', 'rating',
       'review_title', 'country', 'date', 'specs', 'first_spec', 'review',
       'helpful', 'picture', 'verified', 'profile', 'video', 'scrape_date',
       'profile_url', 'product_title_clean'],
      dtype='object')

In [32]:
all_reviews = []

#filter for dvd_player
links_data_filtered = links_data[links_data['category'] == 'smartphone']
links_so_far = data_so_far['asin_url'].unique()
print(len(links_data_filtered))
#remove last digits in links_so_far
links_so_far = [re.sub(r'\d+$', '', link) for link in links_so_far]
links_left = [links for links in links_data_filtered['product_link'] if links not in links_so_far]
print(len(links_left))

436
229


In [18]:
links_data['category'].unique()

array(['dvd_player', 'Digital Photo Frames', 'smartphone',
       'computer adapter', 'printer', 'electric toothbrush', 'mascara',
       'lipstick', 'party dress', 'microwave oven', 'vacuum cleaner',
       'razor'], dtype=object)

In [33]:
#set random seed
import random
random.seed(42)
#shuffle links
random.shuffle(links_left)


In [38]:
l = 1
scrape_links = links_left[100:200]
for link in scrape_links:
    print(f"{l} / {len(scrape_links)}")
    product_info = {
    'product_url' : "https://www.amazon.com/dp/"+link.split('/')[-2],
    'brand' : None,
    'model' : None,
    'link' : link,
    'title' : link.split('/')[3],
    'product_title' : None
    }
    get_reviews_from_page(product_info, 1, 13)
    l += 1
    if l % 10 == 0:
        print('restart container')
        container.restart()
        time.sleep(10)
container.stop()

1 / 100
succesfully loaded Smartphone-BV9300Pro-Phone-Waterproof-BlackGold
page successfully loaded in time 0.2: https://www.amazon.com/Smartphone-BV9300Pro-Phone-Waterproof-BlackGold/product-reviews/B0CW1RJ5LV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
3624
the following items were missing: {'brand': 4, 'model': 4, 'color': 4, 'profile_picture': 3, 'video': 4, 'profile_url': 4, 'helpful': 2}
page successfully loaded in time 0.2: https://www.amazon.com/Smartphone-BV9300Pro-Phone-Waterproof-BlackGold/product-reviews/B0CW1RJ5LV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
Could not find any reviews at page 2
2 / 100
succesfully loaded aderroo-Smartphone%EF%BC%8CR4Pro%EF%BC%8CAndroid-Screen%EF%BC%8CFront-Cameras%EF%BC%8C1GB-ROM%EF%BC%8CSupports
page successfully loaded in time 0.2: https://www.amazon.com/aderroo-Smartphone%EF%BC%8CR4Pro%EF%BC%8CAndroid-Screen%EF%BC%8CFront-Cameras%EF%BC%8C1GB-ROM%EF%BC%8CSupports/product-reviews/

# product info

In [6]:
def get_product_info(url):
    soup_product = get_soup(url)
    product_info = {
    'url' : url
    }
    try:
        product_info['brand'] = soup_product.find('tr', class_ ="a-spacing-small po-brand").find(class_="a-span9").text
    except:
        product_info['brand'] = None
    try:
        product_info['model'] = soup_product.find('tr', class_ ="a-spacing-small po-model_name").find(class_="a-span9").text
    except:
        product_info['model'] = None
    try:
        product_info['title'] = soup_product.title.text.replace('Amazon.com: ','').strip()
    except:
        product_info['title'] = None
    try:
        price = soup_product.find('div', class_="a-section a-spacing-none aok-align-center aok-relative").find("span", class_="aok-offscreen").text
        #extract integers that appear directly after the dollar sign, including after decimal
        price = float(re.search(r'\$(\d+\.\d+)', price).group(1))
        product_info['price'] = price
    except:
        product_info['price'] = None
        #raise Exception('price not found')
    try:
        product_info['rating'] = float(soup_product.find('span', class_="a-icon-alt").text.split(' ')[0])
    except:
        product_info['rating'] = None
    try:
        product_info['reviews'] = int(soup_product.find('span', id="acrCustomerReviewText").text.split(' ')[0].replace(",",""))
    except:
        if soup_product.title.text.strip() == "Page Not Found":
            product_info['reviews'] = None
        else:
            raise Exception('reviews not found')
    try:
        product_info['list_price'] = soup_product.find('span', class_="a-size-small aok-offscreen").text.strip()
    except:
        product_info['list_price'] = None
    try:
        imgs = soup_product.find('div', class_ = "a-fixed-left-grid-col a-col-left").find_all('img')
        pics = [img['src'].replace('_US40_.jpg', '_SX466_.jpg') for img in imgs if img['src'].startswith('https://m.media-amazon.com')]
        product_info['pictures'] = pics
    except:
        product_info['pictures'] = None
    try:
        product_info['description'] = soup_product.find('div', id="productDescription").text.strip()
    except:
        product_info['description'] = None
    return product_info

In [36]:
#make new csv file
#with open(f"../../data/amazon_product_info_per_category.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["url", "brand", "model", "title", "price", "rating", "reviews", "list_price", "pictures", "description"])

In [7]:
data = pd.read_csv(f"../../data/amazon_search_hedonic.csv", delimiter = ";")

C:\Users\janva\AppData\Local\Temp\ipykernel_20356\1172591121.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f"../../data/amazon_search_hedonic.csv", delimiter = ";")


In [8]:
product_info_data = pd.read_csv(f"../../data/amazon_product_info_per_category.csv", delimiter = ";")

In [9]:
def get_asin_url(url):
    asin = url.split('/')[5]
    product_url = "https://www.amazon.com/dp/"+asin
    return product_url

#apply fucntion to asin_url column
asin_urls = data['asin_url'].apply(get_asin_url).unique()
already_done = product_info_data[product_info_data['reviews'].notnull()]['url']
print(len(asin_urls))
print(len(already_done))
#get asin urls that are not in already_done
urls_left = [url for url in list(asin_urls) if url not in list(already_done)]
print(len(urls_left))

1228
150
1078


In [28]:
#container.start()
#time.sleep(10)
i = 0
with open('../../data/amazon_product_info_per_category.csv', 'a', encoding = "UTF-8", newline = '') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter = ";")
    for url in urls_left[0:50]:
            #try:
            product_info = get_product_info(url)
            csv_writer.writerow(list(product_info.values()))
            i += 1
            print(f"{i} / {len(urls_left)}")
            if i % 25 == 0:
                #restart docker
                container.restart()
                time.sleep(30)
            #except:
                 #print(f"error with {url}")
container.stop()

Exception: Captcha

# Pictures

In [57]:
#get a list of all the file names in profile_photos_category
import os
profile_photos = os.listdir("../../gen/output/profile_photos_category")

In [58]:
data = pd.read_csv(f"../../data/amazon_search_hedonic.csv", delimiter = ";")
profile_photos

['0002f3da-1303-4ad6-8b6e-fbd809a866a4._CR80,0,340,340_UX460_.jpg',
 '00059cf5-827d-488c-b8fe-a02e09f64e90._CR0,0,500,500_UX460_.jpg',
 '000750a9-c34e-4865-becc-7c8a5bc8a2f0._CR0,0,375,375_UX460_.jpg',
 '00096ca4-fb0c-47a5-b93a-5e5edeb9940f._CR0,0,100,100_UX110_.jpg',
 '000cdeeb-b7b4-4063-8ab6-456122b0344b._CR0,0,375,375_UX460_.jpg',
 '000cfcf9-7fb9-4ee6-84c1-238a26524e70._CR0,0,375,375_UX460_.jpg',
 '001357ec-6a49-4a02-9e48-0431a3e106ca._CR0,0,541,541_UX460_.jpg',
 '002212d6-3ad3-4537-8329-2b3f4b9e6454._CR62,0,375,375_UX460_.jpg',
 '00263b57-7398-4e50-b703-32af0a04a07f._CR62,0,375,375_UX460_.jpg',
 '002cc6b2-8f40-4fc8-8f86-99461213d94e._CR0,0,768,768_UX460_.jpg',
 '002d2d84-5e1d-4820-9f40-bd00c22d4f61._CR57,0,395,395_UX460_.jpg',
 '002da739-1844-4c3a-8a17-f6a83830de8e._CR0,12.0,308,308_UX460_.jpg',
 '0038634a-843c-40ee-9ce7-d97953a347a6._CR0,0,750,750_UX460_.jpg',
 '00453090-1ae3-48c3-bced-eb52bd94be91._CR0,0,375,375_UX460_.jpg',
 '004809b9-4286-4031-b60f-2ac4f1d576d7._CR62,0,375,375_

In [59]:
scraped_pf = data['profile'].unique()
#return all scraped_pf that are not in profile_photos
pf_left = [pf for pf in scraped_pf[1:] if pf.split('/')[-1] not in profile_photos]

In [56]:
#download pf_left
for pf in pf_left:
    try:
        r = requests.get(pf)
        with open(f"../../gen/output/profile_photos_category/"+pf.split('/')[-1], "wb") as f:
            f.write(r.content)
        time.sleep(0.1)
    except:
        print(f"error with {pf}")

In [60]:
len(pf_left)

0

## consumer photos

In [77]:
consumer_photos = data['picture'].unique()[1:]

In [78]:
import ast
def get_links(pics):
    list_pics = ast.literal_eval(pics)
    return [pic.split('/')[-1] for pic in list_pics]

#apply to consumer)photos
consumer_photos_clean = [get_links(pics) for pics in consumer_photos]
consumer_photos_clean = [item for sublist in consumer_photos_clean for item in sublist]

In [68]:
len(consumer_photos_clean)

26187

In [81]:
#get all existing consumer photos
consumer_photos = os.listdir("../../gen/output/consumer_photos_category")
cp_left = [cp for cp in consumer_photos_clean if cp.replace('_SY88.jpg', '_SL1600_.jpg') not in consumer_photos]
cp_left

[]

In [80]:
#scrape consumer photos
for cp in cp_left:
    try:
        cp = cp.replace('_SY88.jpg', '_SL1600_.jpg')
        r = requests.get("https://images-na.ssl-images-amazon.com/images/I/"+cp)
        with open(f"../../gen/output/consumer_photos_category/"+cp, "wb") as f:
            f.write(r.content)
        time.sleep(0.1)
    except:
        print(f"error with {cp}")